#4-1 線形回帰

## 概要

**線形回帰は、一つ以上の説明変数を使用して、連続値の目的変数を予測するための手法です。**

説明変数とは、「目的変数の原因となっている変数」です。目的変数とは、「予測したい値」です。
一つの説明変数から目的変数を求める方法を単回帰分析といいます。
単回帰分析のモデルは、以下の数式で表されます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_01_01.jpg?raw=true" width="120px">

y が目的変数、x が説明変数、w0、w1は係数です。
2 つ以上の説明変数から目的変数を求める方法は、重回帰分析といいます。
単回帰分析の例を見てみましょう。
体重と身長のデータを散布図として描写し、次に、線形回帰によるモデル化を行うと以下の図のような直線が得られます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_01_02.jpg?raw=true" width="600px">

この直線は、以下の数式となります。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_01_03.jpg?raw=true" width="180px">

これより体重60 kg の人は、一般的に身長約165 cm ということが予測されます。


## OECDとIMFのデータを用いた線形モデルの実装
例として、お金によって人が幸せになるかどうかを知りたいと思い、OECD のウェブサイトから"Better Life Index"内の"Life satisfaction"（暮らしへの満足度）、IMF のウェブサイトから"GDP per capita"（1 人あたりGDP）をダウンロードして、可視化してみます。

In [0]:
#データの取り込み
!wget -nc /content 'https://raw.githubusercontent.com/t-date/DataScience/master/data/oecd_bli_2015.csv'
!wget -nc /content 'https://raw.githubusercontent.com/t-date/DataScience/master/data/gdp_per_capita.csv'

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
# データをロードする
oecd_bli = pd.read_csv("oecd_bli_2015.csv", thousands=",")
gdp_per_capita =pd.read_csv("gdp_per_capita.csv",thousands=",",delimiter="\t", encoding="latin1", na_values="n/a")

In [0]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]

In [0]:
# データを準備する
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

In [0]:
# データを可視化する
country_stats.plot(kind="scatter", x="GDP per capita", y="Life satisfaction")
plt.show()

線形モデルを実装してみます。

In [0]:
# 線形モデルを選択する
from sklearn import linear_model as lm
model = lm.LinearRegression()

In [0]:
# モデルを訓練する
model.fit(X, y)

In [0]:
# キプロスの例から予測を行う
X_new = [[22587]] # キプロスの1 人あたりGDP
print(model.predict(X_new)) # 出力[[ 5.96242338]]

予測モデルを引いてみます。

In [0]:
country_stats.plot(kind="scatter", x="GDP per capita", y="Life satisfaction")
plt.plot(X, model.predict(X))